## Updating our NER

1. Load the model
spacy.load('en')
2. Disable existing pipeline
spacy.blank('en')
3. Add entity recognizer to pipeline
4. Shuffle and loop over the examples
5. update the model(nlp.update)
6. Save the trained model(nlp.to_disk)
7. Test the model

In [1]:
# Load the packages
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm

In [2]:
nlp1 = spacy.load('en')

In [3]:
docx1 = nlp1(u"Who was Kofi Annan?")

In [4]:
for ent in docx1.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Kofi Annan 8 18 PERSON


In [5]:
docx2 = nlp1(u"Who is Steave Jobs?")

In [6]:
for ent1 in docx2.ents:
    print(ent1.text, ent1.start_char, ent1.end_char, ent1.label_)

Steave Jobs 7 18 PERSON


In [7]:
docx3 = nlp1(u"Who is Shaka Khan?")

In [8]:
for ent1 in docx3.ents:
    print(ent1.text, ent1.start_char, ent1.end_char, ent1.label_)

Shaka Khan 7 17 PERSON


In [9]:
TRAIN_DATA = [
    ("Who is Kofi Annan?",{
        "entities":[(8,18,'PERSON')]
    }),
    ("Who is Steve Jobs?",{
        "entities":[(7,17,'PERSON')]
    }),
    ("I like London and Berlin",{
        "entities":[(7,13,'LOC'),(18,24,'LOC')]
    })
]

In [10]:
#Define variables
model = None
output_dir = "F:\Practice"
n_iter = 100

### Load the model

In [11]:
if model is not None:
    nlp = spacy.load(model) #Load existing spacy model
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en') #Create blank Language class
    print("Created blank 'en' model")

Created blank 'en' model


### Set Up the Pipeline

In [12]:
# Create a built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spacy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

### Train the recognizer

1. Add Lables, Annotate them
2. Pipes
3. Begin_training()

In [13]:
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])
        
# get names of other pipes to disable them during the training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):
    optimizer  = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            nlp.update(
                [text],
                [annotations],
                drop=0.5,
                sgd = optimizer,
                losses=losses
            )
            print(losses)

F:\conda_env\lib\site-packages\spacy\language.py:639: UserWarning: [W033] Training a new parser or NER using a model with no lexeme normalization table. This may degrade the performance of the model to some degree. If this is intentional or the language you're using doesn't have a normalization table, please ignore this warning. If this is surprising, make sure you have the spacy-lookups-data package installed. The languages with lexeme normalization tables are currently: da, de, el, en, id, lb, pt, ru, sr, ta, th.
  **kwargs
  0%|                                                                                            | 0/3 [00:00<?, ?it/s]F:\conda_env\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Who is Kofi Annan?" with entities "[(8, 18, 'PERSON')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldPar

{'ner': 2.233333185315132}
{'ner': 6.644745662808418}
{'ner': 11.10265327990055}
{'ner': 4.1645538210868835}
{'ner': 6.163050480186939}


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

{'ner': 10.247451968491077}
{'ner': 3.770753860473633}
{'ner': 5.493203289806843}
{'ner': 8.845895953476429}
{'ner': 3.835399806499481}


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

...


{'ner': 3.0352563153343603e-05}
{'ner': 3.0353484023947786e-05}
{'ner': 9.052883734912892e-14}
{'ner': 6.038578806781103e-08}
{'ner': 6.046117045886386e-08}

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


{'ner': 1.88099904979728e-09}
{'ner': 1.903530307846217e-09}
{'ner': 0.0018577277623803028}
{'ner': 0.004157325069933894}
{'ner': 0.004157349347276583}

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 23.32it/s]


{'ner': 0.0041573526415310535}
{'ner': 4.564819029361795e-08}
{'ner': 4.5648329345113326e-08}
{'ner': 4.570592439388631e-08}


### Test the trained model

In [14]:
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    print("tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('Kofi Annan', 'PERSON')]
tokens [('Who', '', 2), ('is', '', 2), ('Kofi', 'PERSON', 3), ('Annan', 'PERSON', 1), ('?', '', 2)]
Entities [('London', 'LOC'), ('Berlin', 'LOC')]
tokens [('I', '', 2), ('like', '', 2), ('London', 'LOC', 3), ('and', '', 2), ('Berlin', 'LOC', 3)]
Entities [('Steve Jobs', 'PERSON')]
tokens [('Who', '', 2), ('is', '', 2), ('Steve', 'PERSON', 3), ('Jobs', 'PERSON', 1), ('?', '', 2)]


### Save the model

In [15]:
if output_dir is not None:
    output_dir = Path("F:\Practice\spacy_model")
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to ", output_dir)

Saved model to  F:\Practice\spacy_model


### Test the saved model

In [16]:
print("Loading from ", output_dir)
nlp2 = spacy.load(output_dir)
for text, _ in TRAIN_DATA:
    doc = nlp2(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    print("tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    

Loading from  F:\Practice\spacy_model
Entities [('Kofi Annan', 'PERSON')]
tokens [('Who', '', 2), ('is', '', 2), ('Kofi', 'PERSON', 3), ('Annan', 'PERSON', 1), ('?', '', 2)]
Entities [('London', 'LOC'), ('Berlin', 'LOC')]
tokens [('I', '', 2), ('like', '', 2), ('London', 'LOC', 3), ('and', '', 2), ('Berlin', 'LOC', 3)]
Entities [('Steve Jobs', 'PERSON')]
tokens [('Who', '', 2), ('is', '', 2), ('Steve', 'PERSON', 3), ('Jobs', 'PERSON', 1), ('?', '', 2)]
